In [1]:
import tensorflow as tf
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:

mat = loadmat("C:/Users/alexa/Downloads/f1data.mat")

r = range(0,20000,1)

# x    = mat['x'][r][:,0]
# y    = mat['y'][r][:,0]
# tail = mat['tail'][r][:,0]


x    = mat['x'][:,0]
y    = mat['y'][:,0]
tail = mat['tail'][:-1,0]


# print(mat)
print(x.shape)
print(y.shape)
print(tail.shape)

# plt.figure(figsize=(30,5))
# plt.plot(tail)
# plt.show()



(9760224,)
(9760224,)
(9760224,)


In [3]:
# time = np.arange(0, 2000, 0.1);
# data = pd.DataFrame({'x1': np.sin(time),'x2': np.cos(time)})
data = pd.DataFrame({'x': x,'y': y,'tail': tail})

In [4]:
WINDOW_SIZE = 500
OUTPUT_SIZE = 50
BATCH_SIZE  = 100

# All to All

In [ ]:
class CustomGenerator(tf.keras.utils.Sequence):
    def __init__(self, data , input_window_size , output_window_size,batch_size):
        self.batch_size = batch_size
        self.data = data
        self.window_size = input_window_size
        self.output_size = output_window_size
      
    def __len__(self):
        return 100

    def __getitem__(self, index):
        # [batch, timesteps, feature]
        x = np.zeros((self.batch_size,self.window_size,self.data.shape[1]),dtype=np.float32)
        y = np.zeros((self.batch_size,self.output_size,self.data.shape[1]),dtype=np.float32)

        for i in range(index,index+self.batch_size):
            in_start  = i
            in_stop   = in_start + self.window_size
            out_start = in_stop
            out_stop  = out_start + self.output_size
            
            
            print(in_start,in_stop,out_start,out_stop)

            x[i-index,:,:] = self.data[in_start:in_stop]
            y[i-index,:,:] = self.data[out_start:out_stop]

        return x,y

#     def on_epoch_end(self):
#         self.index = np.arange(len(self.indices))
#         if self.shuffle == True:
#             np.random.shuffle(self.index)


In [ ]:
def CreateLSTMModel(window_size,output_size,features):
    model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(window_size,features)),
    tf.keras.layers.LSTM(15, return_sequences=False),
    tf.keras.layers.Dense(output_size*features,kernel_initializer=tf.initializers.zeros),
    tf.keras.layers.Reshape([output_size, features])
    ])

    model.compile(loss=tf.losses.MeanSquaredError(),
                    optimizer=tf.optimizers.Adam(),
                    metrics=[tf.metrics.MeanAbsoluteError()])

    model.summary()

    return model


In [ ]:
model = CreateLSTMModel( window_size = WINDOW_SIZE,
                         output_size = OUTPUT_SIZE,
                         features = data.shape[1])


gen =CustomGenerator(data,WINDOW_SIZE,OUTPUT_SIZE,BATCH_SIZE)


model.fit(gen,epochs= 500,verbose=2)

# Tail To Pos

In [5]:
class TailToPosGenerator(tf.keras.utils.Sequence):
    def __init__(self, data , input_window_size , output_window_size,batch_size):
        self.batch_size = batch_size
        self.data = data
        self.window_size = input_window_size
        self.output_size = output_window_size
        self.on_epoch_end()
        
        self.x = np.zeros((self.batch_size,self.window_size,1),dtype=np.float32)
        self.y = np.zeros((self.batch_size,self.output_size,2),dtype=np.float32)
      
    def __len__(self):
        return 5000

    def __getitem__(self, index):
        # [batch, timesteps, feature]

        idx = self.index[index]
        
        for i in range(idx,idx+self.batch_size):
            in_start  = i
            in_stop   = in_start + self.window_size
            out_start = in_stop
            out_stop  = out_start + self.output_size

            self.x[i-idx,:,0] = self.data["tail"][in_start:in_stop]
            self.y[i-idx,:,:] = self.data[{"x","y"}][out_start:out_stop]
        

        return self.x,self.y

    def on_epoch_end(self):
        self.index = idx = np.random.randint(low=0,high=data.shape[0] - self.batch_size , size=5000)


In [7]:
def TailToPosLSTMModel(window_size,output_size):
    model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(window_size,1),name='inpt'),
    tf.keras.layers.LSTM(32, return_sequences=False,name='lstm'),
    tf.keras.layers.Dense(output_size*2,kernel_initializer=tf.initializers.zeros,name='dense'),
    tf.keras.layers.Reshape([output_size, 2],name='reshape')
    ],name='full_net')

    model.compile(loss=tf.losses.MeanSquaredError(),
                    optimizer=tf.optimizers.Adam(),
                    metrics=[tf.metrics.MeanAbsoluteError()])

    model.summary()

    return model

In [ ]:
model = TailToPosLSTMModel( window_size = WINDOW_SIZE, output_size = OUTPUT_SIZE)

gen =TailToPosGenerator(data,WINDOW_SIZE,OUTPUT_SIZE,BATCH_SIZE)

model.fit(gen,epochs= 500,verbose=1)

Model: "full_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32)                4352      
_________________________________________________________________
dense (Dense)                (None, 100)               3300      
_________________________________________________________________
reshape (Reshape)            (None, 50, 2)             0         
Total params: 7,652
Trainable params: 7,652
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
2022/5000 [===========>..................] - ETA: 3:01:44 - loss: 0.0723 - mean_absolute_error: 0.1333

# Plots

In [ ]:
t = np.zeros((1,WINDOW_SIZE,3),dtype=np.float32)
ty = np.zeros((1,OUTPUT_SIZE,3),dtype=np.float32)

start  = 134
t[0,:,:] = data[start:start+WINDOW_SIZE]
ty[0,:,:] = data[start+WINDOW_SIZE:start+WINDOW_SIZE+OUTPUT_SIZE]

y  = model(t)

print(t.shape,y.shape)

In [ ]:
x = np.asarray(range(window_size))
xx = np.asarray(range(window_size,window_size + output_size))


N = data.shape[1]
for i in range(N):
    plt.figure(figsize=(30,2))
    plt.suptitle(data.columns[i], fontsize=20)
    plt.plot(x,t[0,:,i])
    plt.plot(xx,y[0,:,i],color='green')
    plt.plot(xx,ty[0,:,i],color='red')
plt.show()

In [ ]:
x = np.asarray(range(WINDOW_SIZE))
xx = np.asarray(range(WINDOW_SIZE,WINDOW_SIZE + OUTPUT_SIZE))


for i in range(2):
    plt.figure(figsize=(30,2))
    plt.plot(x,  t[0,:,i])
    plt.plot(xx,ty[0,:,i],color='green')
    plt.plot(xx, y[0,:,i],color='red')
    plt.show()




In [ ]:
prevx = np.cumsum(np.squeeze(t[0,:,0]))
prevy = np.cumsum(np.squeeze(t[0,:,1]))

trux = prevx[-1] + np.cumsum(np.squeeze(ty[0,:,0]))
truy = prevy[-1] + np.cumsum(np.squeeze(ty[0,:,1]))

predx = prevx[-1] + np.cumsum(np.squeeze())
predy = prevy[-1] + np.cumsum(np.squeeze(y[0,:,1]))


plt.plot(prevx,prevy,color='blue')
plt.plot(trux,truy,color='green')
# plt.plot(predx,predy,color='red')


plt.show()